# Sentinel-3 L2 OLCI datasets

Let's monitor changes into OLCI variables of interest over a common geographical area. The datasets are extracted from S3 L2 OLCI _Land_ and _Water_ Full Resolution products using the `xarray` Python package, re-arranged and concatenated together into a final 3d data array which depth is the new _time_ dimension.

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser("~"),"CLEOPE/Trials/modules"))
import qm
import data_processing_S3_olci as dp
import hvplot.xarray

You can choose to work with local files or using the Advanced API. Functions in this tutorial can manage both choices independently and automatically, but only if product of interest are provided (as a list or a file). <br> 
Please note that products used in this example notebook could be *offline*: use the `order` function in the `qm` module to retrieve them first.
### Using the Advanced API (ENS)
The input list must contain the position of products in the remote file system. Note that this list is automatically provided using the `SEARCH.ipynb` trial notebook, aimed at facilitating the data access via ENS. Products are found via ENS at their pseudopath, accessed and processed in situ, without any local download. <br>
### Using OData API
Load a custom list containing the products downloaded in the `local_files` folder. To do that, use the `download` function in the `qm` module, which will download and save items via your own Jupyter Notebook using OData protocol. Check out `DISCOVER_ONDA.ipynb` for more details. <br>
Downloads are stored in the 📁`local_files` folder. The 📄`list_local.txt` file will be created and updated in this working directory with the new downloaded item, authomatically.

In [ ]:
# define the input product list here
files = dp.products("list_WFR.txt")

👉 OLCI L2 variables are collected in the `switcher` dictionary variable from the `dp` module, but keys can be assigned depending on the OLCI product type itself, as shown in the table below:

| _LFR_ | _WFR_ |
| --- | --- |
| `OTCI` OLCI Terrestrial Clorophyll Index| `TSM_NN` Total suspended matter concentration (Neural Net)|
| `OGVI` OLCI Global Vegetation Index | `CHL_OC4ME` Algal pigment concentration|
| `IWV` Integrated water vapour column above the current pixel|`CHL_NN` Algal pigment concentration (Neural Net)|
||`IWV` Integrated water vapour column above the current pixel|

Here below the data set is created on the variable of interest given as input. Optionally set argument `bounds=(xmin,xmax,ymin,ymax)` in function `make_ds` to create a subsample.

In [ ]:
v = "TSM_NN" # variable of interest here compliant to the product type (WFR or LFR)
ds = dp.make_ds(sorted(files),key=v,bounds=(6.306006,14.847897,42.79847,46.531617))  

## Dynamical Map Visualisation

In this example we show two WFR products over Italy, the first sensed before the COVID-19 lockdown, the second one after a few weeks. Majour changes in the TSM are interesting the port areas (e.g. Genoa), rivers and their mouths (e.g. Po and Delta Po mouth) and lakes (e.g. Trasimeno).<br>
⚠️ RAM consuming visualisation, *upgrade CLEOPE computational resources to benefit of increased processing potentialities*.

In [ ]:
import cartopy.crs as ccrs

im_0 = ds.isel(time=0).hvplot.quadmesh(x='lon',y='lat',z=v,cmap="rainbow",clabel=ds[v].attrs["units"],
                                       title=str(ds.coords["time"].data[0]),rasterize=True,
                                       crs=ccrs.PlateCarree(),projection=ccrs.PlateCarree(),
                                       project=True,geo=True,coastline="50m",width=400, height=300)

im_1 = ds.isel(time=1).hvplot.quadmesh(x='lon',y='lat',z=v,cmap="rainbow",clabel=ds[v].attrs["units"],
                                       title=str(ds.coords["time"].data[0]),rasterize=True,
                                       crs=ccrs.PlateCarree(),projection=ccrs.PlateCarree(),
                                       project=True,geo=True,coastline="50m",width=400, height=300)
(im_0+im_1).cols(1)